# VencoPy Tutorial 2

This tutorial aims to give a more in depth overview into the dataParser class and showcases some features that can be customised.

In [ ]:
import sys
import os
from os import path
import pandas as pd
from pathlib import Path
import pathlib
#from ruamel.yaml import YAML

sys.path.append(path.dirname(path.dirname(path.dirname(Path.cwd()))))

from vencopy.classes.dataParsers import DataParser
from vencopy.scripts.globalFunctions import loadConfigDict, createOutputFolders

print("Current working directory: {0}".format(os.getcwd()))

In [ ]:
configNames = ('globalConfig', 'localPathConfig', 'parseConfig', 'tripConfig', 'gridConfig', 'flexConfig', 'evaluatorConfig')
basePath = Path.cwd().parent.parent
configDict = loadConfigDict(configNames, basePath)
createOutputFolders(configDict=configDict)
    
# Set reference dataset 
datasetID = 'MiD17'

# Modify the localPathConfig file to point to the .csv file in the sampling folder in the tutorials directory where the dataset for the tutorials lies.
configDict['localPathConfig']['pathAbsolute'][datasetID] = Path.cwd().parent / 'data_sampling'

# Similarly we modify the datasetID in the global config file
configDict['globalConfig']['files'][datasetID]['tripsDataRaw'] = datasetID + '.csv'

# Adapt relative paths in config for tutorials
configDict['globalConfig']['pathRelative']['plots'] = Path.cwd().parent.parent / configDict['globalConfig']['pathRelative']['plots']
configDict['globalConfig']['pathRelative']['parseOutput'] = Path.cwd().parent.parent / configDict['globalConfig']['pathRelative']['parseOutput']
configDict['globalConfig']['pathRelative']['diaryOutput'] = Path.cwd().parent.parent / configDict['globalConfig']['pathRelative']['diaryOutput']
configDict['globalConfig']['pathRelative']['gridOutput'] = Path.cwd().parent.parent / configDict['globalConfig']['pathRelative']['gridOutput']
configDict['globalConfig']['pathRelative']['flexOutput'] = Path.cwd().parent.parent / configDict['globalConfig']['pathRelative']['flexOutput']
configDict['globalConfig']['pathRelative']['evalOutput'] = Path.cwd().parent.parent / configDict['globalConfig']['pathRelative']['evalOutput']

# We also modify the parseConfig by removing some of the columns that are normally parsed from the MiD, which are not available in our semplified test dataframe
del configDict['parseConfig']['dataVariables']['hhID']
del configDict['parseConfig']['dataVariables']['personID']


## DataParser config file

The DataParser config file defines which variables are to be parsed (i.e. the ones needed to create trip diaries and calculate fleet flexibility) and sets some filtering options, such as the conditions for trips to be included of excluded from the parsing.

<div class="alert alert-block alert-danger"><b>Warning:</b> The list is very long.</div>

In [ ]:
configDict['parseConfig']

## _DataParser_ class

Let's first run the class and see the outputs we get.

In [ ]:
vpData = DataParser(datasetID=datasetID, configDict=configDict, loadEncrypted=False)
vpData.process()
vpData.data.head()

We can see from the print statements in the class that after reading in the initial dataset, which contained 2124 rows, and applying 8 filters, we end up with a database containing 950 suitable entries, which corresponds to about 45% of the initial sample.
These trip respect the condition that they all need to be shorter than 1000km, which is set in the parseConfig under the 'filterDict' key.

Now we can, for example, change in the filters the maximum allowed trip distance from 1000km to 50km and see how this affects the resulting available trips (the extreme case of 50km is only used for the tutorial purpose).

In [ ]:
configDict['parseConfig']['filterDicts']['MiD17']['smallerThan']['tripDistance'] = [50]
configDict['parseConfig']['filterDicts']['MiD17']['smallerThan']['tripDistance']

In [ ]:
vpData = DataParser(datasetID=datasetID, configDict=configDict, loadEncrypted=False)
vpData.process()
vpData.data.head()

We can see how with a maximum trip distance of 1000km, all filters combined yielded a total of 950 trips, which corresponds to about 45% of the original dataset. By changing this values to 50km, additional 36 trips have been excluded, resulting in 914 trips (43% ofthe initial dataset).

## Next Steps

In the next tutorial, you will learn more in detail the internal workings of the TripDiaryBuilder class and how to customise some settings.